In [30]:
import numpy as np
import pandas as pd
import functools
import ast
import re

In [94]:
df1 = pd.read_csv("literature/datasets/Online Hate Speech/reddit.csv")
df2 = pd.read_csv("literature/datasets/Online Hate Speech/gab.csv")

In [91]:
df1

,id,text,hate_speech_idx,response,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,1. e8q18lf\n2. \te8q9w5s\n3. \t\te8qbobk\n4. \...,1. A subsection of retarded Hungarians? Ohh bo...,[1],"[""I don't see a reason why it's okay to insult...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1. e9c6naz\n2. \te9d03a5\n3. \t\te9d8e4d\n,"1. > ""y'all hear sumn?"" by all means I live i...",[3],['Persons with disabilities is the accepted te...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1. e84rl2i\n2. \te84w60l\n3. \t\te8544rn\n4. \...,1. wouldn't the defenders or whatever they are...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1. e7kq72n\n2. \te7m24ar\n,1. Because the Japanese aren't retarded and kn...,[1],"[""It's not right for anyone of any gender to b...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1. e7hdgoh\n2. \te7iyj6a\n3. \t\te7j6iho\n4. \...,1. That might be true if we didn't have an exa...,"[2, 3]","[""You shouldn't be bringing up sensitive topic...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5019,1. e90fta8\n2. \te90g8px\n3. \t\te90jlq5\n4. \...,1. Who does she claim as *her people*?\n2. \tI...,[8],['its inappropriate to use the word faggot as ...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5020,1. e9b7mn2\n2. \te9bv6ir\n,1. Here's what's going to happen: Broward is ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5021,1. e7q8ilv\n2. \te7qddqq\n3. \t\te7qstbc\n4. \...,1. Oh boy...so here comes a long venting post ...,"[1, 4]","[""using the word 'nigger' is a direct attack t...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5022,1. dwp8jut\n,"1. OP, stop being a faggot and post videos nex...",[1],"[""using the words 'faggot fuckers' is offensiv...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
def process1(data):
    data = re.sub(r"\d+\.","",data)
    return data.strip()

def process2(data):
    data = re.sub(r'\n',"",data)
    data = re.sub(r'\t',"",data)
    data = re.sub(r"\xa0","",data)
    return data.strip()
    

def process_data(df):
    processed_column = df['text']
    processed_column = processed_column.apply(lambda x:process1(x))
    processed_column = processed_column.apply(lambda x:process2(x))
    df["text"] = processed_column
    return df

In [79]:
def split_sections(text):
    sections = text.split('\n')
    # Check if sections is empty (i.e., if text did not contain any newline characters)
    if not sections:
        return [text]  # Return the original text as a single element list
    else:
        return sections

In [95]:
sections_list1 = df1['text'].apply(split_sections).tolist()
sections_list2 = df2['text'].apply(split_sections).tolist()

In [96]:
len(df1)

5020

In [97]:
class_list1 = df1["hate_speech_idx"].astype("str").tolist()
class_list2 = df2["hate_speech_idx"].astype("str").tolist()

In [101]:
def process_classes(sections_list, class_list):
    final_list = []
    for i,entry in enumerate(sections_list):
        # if i > 4320:
        #     print(i)
        #     print(entry)
        #     print(class_list[i])
        if class_list[i] == "nan":
            final_list.append([0]*len(entry))
        else:
            entry_classes = (np.array(ast.literal_eval(class_list[i])) - 1).tolist()
            temp = [0]*len(entry)
            temp = np.array(temp)
            try:
                temp[entry_classes] = 1
            except IndexError as e:
                final_list.append([0]*len(entry))
            else:
                temp = temp.tolist()
                final_list.append(temp)

    return final_list

In [102]:
final_list1 = process_classes(sections_list1, class_list1)
final_list2 = process_classes(sections_list2, class_list2)

In [119]:
sections_flat1 = [section for sublist in sections_list1 for section in sublist]
final_list_flat1 = [section for sublist in final_list1 for section in sublist]
sections_flat2 = [section for sublist in sections_list2 for section in sublist]
final_list_flat2 = [section for sublist in final_list2 for section in sublist]

In [120]:
df1_new = pd.DataFrame()
df1_new["text"] = sections_flat1
df1_new["class"] = final_list_flat1
process_data(df1_new)
df2_new = pd.DataFrame()
df2_new["text"] = sections_flat2
df2_new["class"] = final_list_flat2
process_data(df2_new)
df2_new

,text,class
0,i joined gab to remind myself how retarded jew...,1
1,,0
2,This is what the left is really scared of. ...,0
3,That literally looks like a monkey. Why are we...,0
4,Dumb Cunt,1
...,...,...
45596,"I was routinely suspended for saying fuck, and...",1
45597,They got me for dropping the cunt bomb... a lot.,1
45598,,0
45599,How absolutely unbelievably fascinating? A Pak...,1


In [128]:
df = df2_new
filtered_df = df[df['class'] == 0]

In [134]:
# Randomly select rows from the filtered DataFrame
filtered_df.sample(n=3)["text"].tolist() 

['', 'oh come on man', '']

In [144]:
df_final = pd.concat([df1_new, df2_new], axis=0)
df_final.replace('', np.nan, inplace=True)
df_final.replace("[deleted]", np.nan, inplace=True)
df_final.replace("[removed]", np.nan, inplace=True)
cleaned_df = df_final.dropna(subset=['text'], axis=0)

In [145]:
df_final

,text,class
0,A subsection of retarded Hungarians? Ohh boy. ...,1
1,Hiii. Just got off work. 444 is mainly the typ...,0
2,wow i guess soyboys are the same in every country,0
3,Owen Benjamin's soyboy song goes for every cou...,0
4,NaN,0
...,...,...
45596,"I was routinely suspended for saying fuck, and...",1
45597,They got me for dropping the cunt bomb... a lot.,1
45598,NaN,0
45599,How absolutely unbelievably fascinating? A Pak...,1


In [146]:
cleaned_df

,text,class
0,A subsection of retarded Hungarians? Ohh boy. ...,1
1,Hiii. Just got off work. 444 is mainly the typ...,0
2,wow i guess soyboys are the same in every country,0
3,Owen Benjamin's soyboy song goes for every cou...,0
5,"> ""y'all hear sumn?"" by all means I live in a...",0
...,...,...
45593,fucking retard,1
45595,Twitter BANNED me and reported me to the FBI f...,0
45596,"I was routinely suspended for saying fuck, and...",1
45597,They got me for dropping the cunt bomb... a lot.,1


In [147]:
cleaned_df.to_csv("literature/datasets/Online Hate Speech/combined.csv",header=True, index=False)